In [1]:
!pip install transformers datasets
!pip install torch


In [2]:
from datasets import load_dataset

# Charger le jeu de données IMDb
dataset = load_dataset('imdb')


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [3]:
from transformers import DistilBertTokenizer

# Charger le tokenizer de DistilBERT
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Tokenisation des textes
def tokenize_data(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

# Appliquer la tokenisation au jeu de données
tokenized_dataset = dataset.map(tokenize_data, batched=True)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [4]:
pip install --upgrade transformers

In [5]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",          # Évaluer à chaque fin d'époque
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test']
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# Entraîner le modèle
trainer.train()

# Évaluer le modèle
trainer.evaluate()


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: exaucebonheu (exaucebonheu-ville-de-lille) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,0.226700,0.221926
2,0.152700,0.228502
3,0.087600,0.281899


{'eval_loss': 0.281898558139801,
 'eval_runtime': 355.2167,
 'eval_samples_per_second': 70.38,
 'eval_steps_per_second': 4.4,
 'epoch': 3.0}

In [7]:
# Sauvegarder le modèle finement ajusté
model.save_pretrained('./fine_tuned_distilbert')
tokenizer.save_pretrained('./fine_tuned_distilbert')


('./fine_tuned_distilbert/tokenizer_config.json',
 './fine_tuned_distilbert/special_tokens_map.json',
 './fine_tuned_distilbert/vocab.txt',
 './fine_tuned_distilbert/added_tokens.json')

In [8]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer

# Recharger le modèle et le tokenizer depuis le dossier où ils ont été sauvegardés
model = DistilBertForSequenceClassification.from_pretrained('./fine_tuned_distilbert')
tokenizer = DistilBertTokenizer.from_pretrained('./fine_tuned_distilbert')


In [9]:
# Exemple de texte à tester
text = "This movie was absolutely bad! I dind'nt love it."

# Tokenisation de l'exemple
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)


In [12]:
import torch  # ← Ajoutez cette importation

texts = [
    "This morning started in a wonderful way because I woke up feeling refreshed and full of energy.",
    "The breakfast was delicious, and the coffee tasted perfect.",
    "On my way to work, however, the heavy traffic made me frustrated and impatient.",
    "I felt disappointed when I realized I had forgotten an important document at home.",
    "Luckily, one of my colleagues was kind enough to help me and share the files I needed.",
    "Later, I received positive feedback on the project I had been working on, which made me proud and happy.",
    "In the afternoon, the internet connection kept failing, and it slowed down my tasks, which was very annoying.",
    "Despite that, the meeting with my team was productive and motivating.",
    "By the end of the day, I was tired, but I felt satisfied with everything I had accomplished."
]

for text in texts:
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():  # ← Maintenant torch est reconnu
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = logits.argmax().item()

    if predicted_class == 1:
        print(f"Texte : '{text}' -> Classe prédite : Positive")
    else:
        print(f"Texte : '{text}' -> Classe prédite : Négative")

Texte : 'This morning started in a wonderful way because I woke up feeling refreshed and full of energy.' -> Classe prédite : Positive
Texte : 'The breakfast was delicious, and the coffee tasted perfect.' -> Classe prédite : Positive
Texte : 'On my way to work, however, the heavy traffic made me frustrated and impatient.' -> Classe prédite : Négative
Texte : 'I felt disappointed when I realized I had forgotten an important document at home.' -> Classe prédite : Négative
Texte : 'Luckily, one of my colleagues was kind enough to help me and share the files I needed.' -> Classe prédite : Positive
Texte : 'Later, I received positive feedback on the project I had been working on, which made me proud and happy.' -> Classe prédite : Positive
Texte : 'In the afternoon, the internet connection kept failing, and it slowed down my tasks, which was very annoying.' -> Classe prédite : Négative
Texte : 'Despite that, the meeting with my team was productive and motivating.' -> Classe prédite : Positi